In [1]:
import numpy as np
import mahotas as mh
from mahotas.features import surf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.cluster import MiniBatchKMeans
import glob

In [2]:
all_instance_filenames = []
all_instance_targets = []

number_images = 1000

data_path = './data/dog-v-cat/train/{}.{}.jpg'

for sp in ['cat', 'dog']:
    for n in range(1, number_images+1):
        target = 1 if sp == 'cat' else 0
        path = data_path.format(sp, n)
        all_instance_filenames.append(path)
        all_instance_targets.append(target)
    
surf_features = []
counter = 0
for f in all_instance_filenames:
    print 'Reading image:', f
    image = mh.imread(f, as_grey=True)
    surf_features.append(surf.surf(image)[:, 5:])

Reading image: ./data/dog-v-cat/train/cat.1.jpg
Reading image: ./data/dog-v-cat/train/cat.2.jpg
Reading image: ./data/dog-v-cat/train/cat.3.jpg
Reading image: ./data/dog-v-cat/train/cat.4.jpg
Reading image: ./data/dog-v-cat/train/cat.5.jpg
Reading image: ./data/dog-v-cat/train/cat.6.jpg
Reading image: ./data/dog-v-cat/train/cat.7.jpg
Reading image: ./data/dog-v-cat/train/cat.8.jpg
Reading image: ./data/dog-v-cat/train/cat.9.jpg
Reading image: ./data/dog-v-cat/train/cat.10.jpg
Reading image: ./data/dog-v-cat/train/cat.11.jpg
Reading image: ./data/dog-v-cat/train/cat.12.jpg
Reading image: ./data/dog-v-cat/train/cat.13.jpg
Reading image: ./data/dog-v-cat/train/cat.14.jpg
Reading image: ./data/dog-v-cat/train/cat.15.jpg
Reading image: ./data/dog-v-cat/train/cat.16.jpg
Reading image: ./data/dog-v-cat/train/cat.17.jpg
Reading image: ./data/dog-v-cat/train/cat.18.jpg
Reading image: ./data/dog-v-cat/train/cat.19.jpg
Reading image: ./data/dog-v-cat/train/cat.20.jpg
Reading image: ./data/dog-v-c